In [10]:
import pandas as pd
import json

with open('intents.json', 'r', encoding='utf-8') as file:
    data = json.load(file)

df = pd.DataFrame(data)

df.head()


,intents
0,"{'tag': 'greeting', 'patterns': ['Hi', 'Hey', ..."
1,"{'tag': 'morning', 'patterns': ['Good morning'..."
2,"{'tag': 'afternoon', 'patterns': ['Good aftern..."
3,"{'tag': 'evening', 'patterns': ['Good evening'..."
4,"{'tag': 'night', 'patterns': ['Good night'], '..."


In [11]:
intent_patterns = []

for intent in data['intents']:
    for pattern in intent['patterns']:
        intent_patterns.append({
            "pattern": pattern,
            "tag": intent['tag'],
            "responses": intent['responses']
        })

df = pd.DataFrame(intent_patterns)
df.head()

,pattern,tag,responses
0,Hi,greeting,[Hello there. Tell me how are you feeling toda...
1,Hey,greeting,[Hello there. Tell me how are you feeling toda...
2,Is anyone there?,greeting,[Hello there. Tell me how are you feeling toda...
3,Hi there,greeting,[Hello there. Tell me how are you feeling toda...
4,Hello,greeting,[Hello there. Tell me how are you feeling toda...


In [12]:
import sentence_transformers
from sentence_transformers import SentenceTransformer
from sentence_transformers import util

model = SentenceTransformer('paraphrase-MiniLM-L6-v2')  # lightweight and good

corpus = [item["pattern"] for item in intent_patterns]
corpus_embeddings = model.encode(corpus, convert_to_tensor=True) 

In [13]:
import random

def get_response(user_input):
    user_embedding = model.encode(user_input, convert_to_tensor=True)
    
    # Find the closest pattern
    scores = util.pytorch_cos_sim(user_embedding, corpus_embeddings)[0]
    best_match_idx = scores.argmax().item()
    
    # Get matched intent
    matched = intent_patterns[best_match_idx]
    response = random.choice(matched['responses'])

    return matched['tag'], response


In [ ]:
while True:
    user_input = input("You: ")
    if user_input.lower() == "quit":
        break
    tag, bot_response = get_response(user_input)
    print(f"[{tag.upper()}] Bot: {bot_response}")


[SUICIDE] Bot: I'm very sorry to hear that but you have so much to look forward to. Please seek help by contacting: 9152987821.
[NOT-TALKING] Bot: I want to help you. I really do. But in order for me to help you, you're gonna have to talk to me.
[WRONG] Bot: I'm very sorry. Let's try that again
[SKILL] Bot: I can provide general advice regarding anxiety and depression, answer questions related to mental health and make daily conversations. Do not consider me as a subsitute for an actual mental healthcare worker. Please seek help if you don't feel satisfied with me.
[FACT-9] Bot: It is estimated that mental illness affects 1 in 5 adults in America, and that 1 in 24 adults have a serious mental illness. Mental illness does not discriminate; it can affect anyone, regardless of gender, age, income, social status, ethnicity, religion, sexual orientation, or background. Although mental illness can affect anyone, certain conditions may be more common in different populations. For instance, ea